In [96]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os

albers_hi = '+proj=aea +lat_0=13 +lon_0=-157 +lat_1=8 +lat_2=18 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs +type=crs'

# Data preparation

In [102]:
maui_zoning = gpd.read_file('data/Zoning_(County_of_Maui).geojson')
maui_parcels = gpd.read_file('data/Parcels_Maui_County.geojson')

# CRS are the same
maui_zoning.crs == maui_parcels.crs

# zoning is already dissolved for 1 row per district

True

In [105]:
# drop unnecessary columns
maui_zoning = maui_zoning[['zone_class', 'cp_area', 'cp_area_name', 'island', 'island_name', 'geometry']]

# reduce to only relevant R districts for speed
maui_zoning = maui_zoning[maui_zoning['zone_class'].str.match(r"^R-[1-3]")]

# dissolve on zone in area
maui_zoning = maui_zoning.dissolve(by=['cp_area', 'zone_class'], aggfunc='first', as_index=False)

In [112]:
maui_zoning.cp_area.value_counts()

cp_area
KM     3
LN     3
MPK    3
PH     3
WK     3
WM     3
ML     2
HN     1
Name: count, dtype: int64

In [113]:
# clip parcels to zones (9m)
maui_parcels = gpd.clip(maui_parcels, maui_zoning)

In [114]:
# add zoning data to R parcels (2.5min)
maui_parcels_in_zones = gpd.sjoin(maui_parcels, maui_zoning, how = 'inner')

In [115]:
print(len(maui_parcels_in_zones))
# FIXME getting more than a 1:1 ratio of parcels to zones
print(round(len(maui_parcels_in_zones)/len(maui_parcels), 2))

22132
1.04


In [116]:
# calculate parcel area
original_crs = maui_parcels_in_zones.crs
maui_parcels_in_zones['geometry'] = maui_parcels_in_zones['geometry'].to_crs(albers_hi)
maui_parcels_in_zones['parcel_area'] = maui_parcels_in_zones.geometry.area
maui_parcels_in_zones['geometry'] = maui_parcels_in_zones['geometry'].to_crs(original_crs)

# convert from sqm to sqft
maui_parcels_in_zones['parcel_area'] = maui_parcels_in_zones['parcel_area'] * 10.7639

In [119]:
maui_parcels_in_zones = maui_parcels_in_zones[['tmk', 'tmk_txt', 'cty_tmk', 'landvalue', 'landexempt',
       'bldgvalue', 'bldgexempt', 'pittcode', 'homeowner', 'nhoodcode',
       'taxacres', 'gisacres', 'qpub_link', 'geometry', 'cp_area', 'zone_class',
       'cp_area_name', 'island', 'island_name', 'parcel_area']]

In [120]:
# spatial join parcels to islands for filtering
maui_parcels_in_zones.to_file('data/maui_gdf.geojson', driver='GeoJSON')

# lanai = maui_islands[maui_islands['island_name' == 'Lanai']]
# molokai = maui_islands[maui_islands['island_name' == 'Molokai']]
# maui = maui_islands[maui_islands['island_name' == 'Maui']]

# Analysis

In [123]:
maui_gdf = gpd.read_file('data/maui_gdf.geojson')

In [152]:
# Highest value properties are >300M. Hotel, UH, etc.
# maui_gdf.nlargest(10,'bldgvalue')['qpub_link'].to_list()

['https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=440140060000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=440140050000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=440130080000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=380010190000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=380460130000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=440130130000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=230080390000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=230080390000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=380070400000',
 'https://qpublic.schneidercorp.com/Application.aspx?AppID=1029&PageTypeID=4&KeyValue=39001

In [180]:
# Assume a lot is occupied if the bldgvalue is > 50000
maui_gdf['occupied'] = maui_gdf['bldgvalue'].apply(lambda x: 1 if (x > 50000) else 0)

# calculate possible number of units under existing law
# https://library.municode.com/hi/county_of_maui/codes/code_of_ordinances?nodeId=TIT19ZO_ARTIICOZOPR_CH19.08REDI_19.08.040DESTHERESELI
# 1 per 6000 R-1, 7500 R-2, 10000 R-3
def zone_sort(zone):
    if zone == 'R-1 Residential':
        return 6000
    if zone == 'R-2 Residential':
        return 7500
    if zone == 'R-3 Residential':
        return 10000

maui_gdf['current_limit'] = maui_gdf['zone_class'].apply(zone_sort)
maui_gdf['current_limit'] = maui_gdf['parcel_area'] // maui_gdf['current_limit']

# calculate possible number of units under new law
# no change for Molokai
maui_gdf['proposed_limit'] = maui_gdf['current_limit']
# parcel area // 2500 (for Lanai and Maui)
maui_gdf.loc[(maui_gdf['island_name'] != 'Molokai'), 'proposed_limit'] = maui_gdf['parcel_area'] // 2500

# calculate gross gain in potential units (without taking into accounts actual units or ADUs)
maui_gdf['gross_increase'] = maui_gdf['proposed_limit'] - maui_gdf['current_limit']

# calculate net gain in potential units (subtracting occupied)
maui_gdf['net_increase'] = maui_gdf['gross_increase'] - maui_gdf['occupied']

In [182]:
# calculate difference on all parcels
sum(maui_gdf['net_increase'])

69324.0

In [185]:
# calculate difference on vacant parcels only
sum(maui_gdf[(maui_gdf['occupied'] == 0)]['net_increase'])

27511.0

In [187]:
# calculate sum as a pct of existing law
round(sum(maui_gdf['proposed_limit']) / sum(maui_gdf['current_limit']),2)

3.76

In [196]:
# crosstab of islands and zones
pd.pivot_table(maui_gdf,
               values = 'net_increase',
               index = 'island_name',
               columns = 'zone_class',
               aggfunc = 'sum',
               margins = True)

# calculate sum as a pct of existing housing stock

zone_class,R-1 Residential,R-2 Residential,R-3 Residential,All
island_name,,,,
Lanai,466.0,94.0,92.0,652.0
Maui,7442.0,18837.0,42410.0,68689.0
Molokai,NaN,0.0,-17.0,-17.0
All,7908.0,18931.0,42485.0,69324.0


In [ ]:
# crosstab of islands and zones
pd.pivot_table(maui_gdf,
               values = 'net_increase',
               index = 'island_name',
               columns = 'zone_class',
               aggfunc = 'median',
               margins = True)

# calculate sum as a pct of existing housing stock

zone_class,R-1 Residential,R-2 Residential,R-3 Residential,All
island_name,,,,
Lanai,0.0,1.0,5.0,0.0
Maui,1.0,1.0,2.0,1.0
Molokai,NaN,0.0,-1.0,-1.0
All,0.0,1.0,2.0,1.0


In [191]:
# crosstab of CP areas and zones
pd.pivot_table(maui_gdf,
               values = 'net_increase',
               index = 'cp_area_name',
               columns = 'zone_class',
               aggfunc = 'sum',
               margins = True)

zone_class,R-1 Residential,R-2 Residential,R-3 Residential,All
cp_area_name,,,,
Hana,NaN,NaN,340.0,340.0
Kihei-Makena,2388.0,6240.0,8730.0,17358.0
Lanai,466.0,94.0,92.0,652.0
Makawao-Pukalani-Kula,1585.0,2747.0,9243.0,13575.0
Molokai,NaN,0.0,-17.0,-17.0
Paia-Haiku,853.0,144.0,1829.0,2826.0
Wailuku-Kahului,816.0,6731.0,9562.0,17109.0
West Maui,1800.0,2975.0,12706.0,17481.0
All,7908.0,18931.0,42485.0,69324.0


In [ ]:
# TODO join back to CP areas and choropleth net increase

In [190]:
# create tabular file for Grassroot
maui_tab = maui_gdf.drop('geometry', axis=1)
maui_tab.to_csv('output/maui_nui_parcels_with_zoning.csv')

In [ ]:
# Potential extensions

# ADUs

# KDA 